# 02 – Generación de Base de Ubicaciones con Métricas de Visitas (filtrando unicamente puntos y ubicaciones dentro del Área Metropolitana de San Salvador)
Asigna una zona geográfica a cada ping, utilizando un shapefile o archivo GeoJSON de polígonos.

In [0]:
%sql
--tabla de pings
select * from sv_12_2023_filtered limit 40

In [0]:
%sql
--tabla de ubicaciones
select * from osm_shapes_filtered limit 40

Prefiltrando ambas tablas con las coordenadas del área metropolitana de San Salvador


In [0]:
from pyspark.sql import functions as F
from shapely.wkt import loads
from pyspark.sql.types import StructType, StructField, DoubleType

AMSS_MIN_LON = -89.35
AMSS_MAX_LON = -89.10
AMSS_MIN_LAT =  13.55
AMSS_MAX_LAT =  13.90

@F.udf(returnType=StructType([
    StructField("minx", DoubleType()),
    StructField("miny", DoubleType()),
    StructField("maxx", DoubleType()),
    StructField("maxy", DoubleType())
]))
def get_bbox_udf(wkt):
    try:
        geom = loads(wkt)
        minx, miny, maxx, maxy = geom.bounds
        return (float(minx), float(miny), float(maxx), float(maxy))
    except:
        return (None, None, None, None)

In [0]:
points_amss = (
    spark.table("sv_12_2023_filtered")
    .filter(
        (F.col("longitude") >= AMSS_MIN_LON) & (F.col("longitude") <= AMSS_MAX_LON) &
        (F.col("latitude") >= AMSS_MIN_LAT) & (F.col("latitude") <= AMSS_MAX_LAT)
    )
)

In [0]:
cnt_prv = spark.table("sv_12_2023_filtered").count()
cnt_pos = points_amss.count()
print(f"Registros previos al filtro {cnt_prv} vs registros posteriores al filtro {cnt_pos}")

Filtrando los poligonos (shapes) que esten dentro del bbox del AMSS

In [0]:
polys = spark.table("osm_shapes_filtered").withColumn("bbox", get_bbox_udf(F.col("geometry_wkt")))

polys_amss_bbox = polys.select(
    "*",
    F.col("bbox.minx").alias("minx"),
    F.col("bbox.miny").alias("miny"),
    F.col("bbox.maxx").alias("maxx"),
    F.col("bbox.maxy").alias("maxy"),
)

polys_amss_bbox = polys_amss_bbox.filter(
    (F.col("maxx") >= AMSS_MIN_LON) & (F.col("minx") <= AMSS_MAX_LON) &
    (F.col("maxy") >= AMSS_MIN_LAT) & (F.col("miny") <= AMSS_MAX_LAT)
)

In [0]:
cnt_prv = spark.table("osm_shapes_filtered").count()
cnt_pos = polys_amss_bbox.count()
print(f"Ubicaciones previas al filtro {cnt_prv} vs ubicaciones posteriores al filtro {cnt_pos}")

Ejecutando join entre ambas tablas, primero por bounding box creando un rectangulo a partir de los maximos y minimos de latitud y longitud de la ubicacion y luego filtrando con la UDF que efectivamente el ping esté dentro de la ubicación, para evitar el cross join que exigiría mucho más computo

In [0]:
points_amss.write.mode("overwrite").saveAsTable("sv_12_2023_filtered_amss")
polys_amss_bbox.write.mode("overwrite").saveAsTable("osm_shapes_filtered_amss")

In [0]:
from pyspark.sql.types import BooleanType
from shapely.geometry import Point

#definiendo UDF para evaluar el punto dentro del polígono

def point_in_polygon_udf(lon, lat, polygon_wkt):
    try:
        poly = loads(polygon_wkt)
        point = Point(lon, lat)
        return poly.contains(point)
    except Exception:
        return False
    
spark.udf.register("point_in_polygon", point_in_polygon_udf, BooleanType())

In [0]:
%sql
CREATE TABLE sv_12_2023_locations_w_pings AS
SELECT /*+ BROADCAST(a) */  -- si a (shapes) es pequeña
  a.osm_id, a.clase, a.nombre, a.geometry_wkt,
  b.datetime, b.hora, b.fecha, b.is_weekend, b.nombre_dia, b.device_id,
  b.latitude, b.longitude
FROM osm_shapes_filtered_amss a
JOIN sv_12_2023_filtered_amss b
  ON b.longitude BETWEEN a.minx AND a.maxx
 AND b.latitude  BETWEEN a.miny AND a.maxy
WHERE point_in_polygon(b.longitude, b.latitude, a.geometry_wkt) = true;


In [0]:
%sql
select * from sv_12_2023_locations_w_pings limit 40

exportando pings para modelar clustering espaciales

In [0]:
df_spark = spark.table("sv_12_2023_filtered_amss").select("device_id", "latitude", "longitude", "timestamp")
df_pandas = df_spark.toPandas()

In [0]:
import math
import csv

chunk_size = 500_000
n_chunks = math.ceil(len(df_pandas) / chunk_size)

for i in range(n_chunks):
    start, end = i * chunk_size, (i + 1) * chunk_size
    chunk = df_pandas.iloc[start:end]
    out_path = f"../datos/filtered_amss/sv_12_2023_filtered_amss_part_{i+1}.csv"
    chunk.to_csv(out_path, encoding='utf-8', index=False, quoting=csv.QUOTE_ALL)
    print(f"✅ Guardado chunk {i+1}/{n_chunks}: {out_path}")